# <center>Advanced MPI</center>


### Stencil Computations

Iterative Stencil Loops (ISLs) or Stencil computations are a class of numerical data processing solution which update array elements according to some fixed pattern, called a stencil.They are most commonly found in computer simulations, e.g. for computational fluid dynamics in the context of scientific and engineering applications. Other notable examples include solving partial differential equations, the Jacobi kernel, the Gauss–Seidel method,image processing and cellular automata. 

The regular structure of the arrays sets stencil techniques apart from other modeling methods such as the Finite element method. Most finite difference codes which operate on regular grids can be formulated as ISLs.

SLs perform a sequence of sweeps (called timesteps) through a given array. Generally this is a 2- or 3-dimensional regular grid. The elements of the arrays are often referred to as cells. In each timestep, all array elements are updated. Using neighboring array elements in a fixed pattern (the stencil), each cell's new value is computed. In most cases boundary values are left unchanged, but in some cases (e.g. LBM codes) those need to be adjusted during the computation as well. Since the stencil is the same for each element, the pattern of data accesses is repeated. Source [Wikipedia](https://en.wikipedia.org/wiki/Iterative_Stencil_Loops)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Von_Neumann_neighborhood.svg/800px-Von_Neumann_neighborhood.svg.png" height=200px width=200px>

## Basic Stencil Operation


![Basic Stencil Operation](./img/stencil_basic.png)

# How to run MPI programs for the stencil codes 

### Step 1: Create and initialize the Grid 

```c++
#define N 10  // Grid size
#define ITERATIONS 100  // Number of iterations

// Function to initialize the grid
void initialize_grid(double grid[N][N]) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            grid[i][j] = rand() % 100;
        }
    }
}
```

### Step 2: Define the stencil Step operation

```c++
// Function to perform the stencil computation
void stencil_step(double grid[N][N], double new_grid[N][N]) {
    for (int i = 1; i < N-1; i++) {
        for (int j = 1; j < N-1; j++) {
            new_grid[i][j] = 0.25 * (grid[i-1][j] + grid[i+1][j] + grid[i][j-1] + grid[i][j+1]);
        }
    }
}
```



### Step 3: Define how to copy the Grid 
```c++
// Function to copy the new grid to the old grid
void copy_grid(double dest[N][N], double src[N][N]) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            dest[i][j] = src[i][j];
        }
    }
}

```


### Step 4: Create the MPI process for running the stencil operation over the ranks

```c++
int main(int argc, char *argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    double grid[N][N], new_grid[N][N];
    initialize_grid(grid);

    for (int iter = 0; iter < ITERATIONS; iter++) {
        stencil_step(grid, new_grid);
        copy_grid(grid, new_grid);

        // Exchange boundary rows between neighboring processes
        if (rank > 0) {
            MPI_Send(grid[1], N, MPI_DOUBLE, rank-1, 0, MPI_COMM_WORLD);
            MPI_Recv(grid[0], N, MPI_DOUBLE, rank-1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
        if (rank < size-1) {
            MPI_Send(grid[N-2], N, MPI_DOUBLE, rank+1, 0, MPI_COMM_WORLD);
            MPI_Recv(grid[N-1], N, MPI_DOUBLE, rank+1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
    }

    MPI_Finalize();
    return 0;
}
```

In [18]:
!mpicc code/basic_stencil.cpp -o basic_stencil.exe

#!srun -N 1 ./basic_stencil.exe
#

In [19]:
!ulimit -s 800000; srun -n 1 ./basic_stencil.exe

The size of this matrix is 10000 
 Initial Grid:
Final Grid:


In [20]:
import pandas as pd 

In [23]:
initial_matrix=pd.read_csv("./initial_grid.txt",  header=None, delim_whitespace=True)

/tmp/ipykernel_1116302/1792226266.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  pd.read_csv("./initial_grid.txt",  header=None, delim_whitespace=True)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,83.0,86.0,77.0,15.0,93.0,35.0,86.0,92.0,49.0,21.0,...,32.0,60.0,76.0,68.0,39.0,12.0,26.0,86.0,94.0,39.0
1,95.0,70.0,34.0,78.0,67.0,1.0,97.0,2.0,17.0,92.0,...,29.0,37.0,35.0,93.0,18.0,28.0,43.0,11.0,28.0,29.0
2,76.0,4.0,43.0,63.0,13.0,38.0,6.0,40.0,4.0,18.0,...,59.0,73.0,13.0,68.0,90.0,95.0,26.0,66.0,84.0,40.0
3,90.0,84.0,76.0,42.0,36.0,7.0,45.0,56.0,79.0,18.0,...,26.0,59.0,44.0,2.0,2.0,6.0,84.0,21.0,42.0,68.0
4,28.0,89.0,72.0,8.0,58.0,98.0,36.0,8.0,53.0,48.0,...,43.0,44.0,29.0,90.0,82.0,40.0,41.0,69.0,26.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8.0,0.0,14.0,51.0,38.0,17.0,51.0,83.0,52.0,61.0,...,48.0,14.0,39.0,29.0,47.0,71.0,13.0,48.0,20.0,51.0
96,4.0,92.0,57.0,14.0,5.0,89.0,1.0,92.0,68.0,81.0,...,35.0,86.0,24.0,3.0,92.0,75.0,97.0,41.0,42.0,74.0
97,25.0,97.0,12.0,0.0,2.0,44.0,14.0,45.0,79.0,4.0,...,1.0,55.0,76.0,19.0,50.0,27.0,10.0,21.0,86.0,36.0
98,64.0,38.0,54.0,10.0,19.0,79.0,75.0,16.0,65.0,16.0,...,71.0,67.0,81.0,60.0,1.0,74.0,77.0,56.0,26.0,0.0


In [ ]:
final_matrix=pd.read_csv("./final_grid.txt",  header=None, delim_whitespace=True)